# Signal vs. background classification with NEW full MC

In this notebook we read in the prepared data, construct and train the DNN, and then evaluate its performance.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy  as np
import random as rd
import tables as tb
import h5py
import tensorflow as tf

from matplotlib.patches         import Ellipse
from __future__  import print_function
from scipy.stats import threshold

# Keras imports
import keras.backend.tensorflow_backend as K
from keras.models               import Model, load_model
from keras.layers               import Input, Dense, MaxPooling3D, AveragePooling3D, Convolution3D, Activation, Dropout, merge
from keras.layers.normalization import BatchNormalization
from keras.optimizers           import SGD, Adam, Nadam         
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core          import Flatten
from keras                      import callbacks
from keras.regularizers         import l2, activity_l2
tf.python.control_flow_ops = tf

# Variable definitions
Here we define key variables to be used throughout the notebook.  Note that we will read the data from a directory `data_location/run_name`, and it is stored in multiple files:
- The training data will consist of the events stored in files from `train_fstart` to `train_fend`
- The test data will consist of the events stored in files from `test_fstart` to `test_fend`

In [ ]:
# data dimensions
xdim = 20
ydim = 20
zdim = 60

# data location and training/test file numbers
data_location = "/home/jrenner/data/classification"
run_name = "1M_v0_08_07"
train_fstart = 0
train_fend = 2
test_fstart = 4
test_fend = 5

## Function definitions

### Data input functions

In [ ]:
# define the function to read the data from multiple files
def read_data(loc, rname, f_start, f_end):
    """Reads all events from the files with the specified file numbers."""
    
    # read in the signal events.
    print("Reading signal events...")
    for fn in range(f_start,f_end):
        s_dat = tb.open_file("{0}/{1}/hdf5_maps_NEW_training_MC_si_{2}.h5".format(loc,rname,fn), 'r')
        if(fn == f_start):
            s_array = np.array(s_dat.root.maps)
            s_energies = np.array(s_dat.root.energies)
            print("-- Reading file {0},".format(fn), end=' ')
        else:
            print("{0},".format(fn), end=' ')
            s_array = np.concatenate([s_array,np.array(s_dat.root.maps)])
            s_energies = np.concatenate([s_energies,np.array(s_dat.root.energies)])
    print("done.")

    # read in the background events.
    print("\nReading background events...")
    for fn in range(f_start,f_end):
        b_dat = tb.open_file("{0}/{1}/hdf5_maps_NEW_training_MC_bg_{2}.h5".format(loc,rname,fn), 'r')
        if(fn == f_start):
            print("-- Reading file {0},".format(fn), end=' ')
            b_array = np.array(b_dat.root.maps)
            b_energies = np.array(b_dat.root.energies)
        else:
            print("{0},".format(fn), end=' ')
            b_array = np.concatenate([b_array,np.array(b_dat.root.maps)])
            b_energies = np.concatenate([b_energies,np.array(b_dat.root.energies)])
    print("done.")
    print("\nRead {0} signal events and {1} background events.".format(len(s_array),len(b_array)))
        
    # concatenate the datasets
    print("Concatenating datasets...", end=' ')
    x_ = np.concatenate([s_array, b_array])
    y_ = np.concatenate([np.ones([len(s_array), 1]), np.zeros([len(b_array), 1])])
    print("done.")
    
    # reshape for training with TensorFlow
    print("Reshaping projection...", end=' ')
    x_ = np.reshape(x_, (len(x_), xdim, ydim, zdim, 1))
    print("done.")
    
    print("Finished reading data.")
    return x_,y_

### Neural network models
These functions should define and return a Keras model object.

In [ ]:
##################################
# Define more neural networks here

def model_(inputs):
    
    # define the network
    
    # ----------
    return model

In [ ]:
# define a fully-connected neural network with 64 hidden neurons and 1 readout neuron
def model_FC(inputs):
    
    f1 = Flatten()(inputs)
    f1 = Dense(output_dim=64, activation='sigmoid', init='normal')(f1)
    f1 = Dropout(.3)(f1)
    inc_output = Dense(output_dim=1, activation='sigmoid', init='normal')(f1)
    model = Model(inputs, inc_output)

    model.compile(loss='binary_crossentropy',
                  optimizer=Nadam(lr=0.00001, beta_1=0.9, beta_2=0.999,
                                  epsilon=1e-08, schedule_decay=0.01),
                                  metrics=['accuracy'])  
    return model

In [ ]:
# plot a 20x20 SiPM map
# -- carried over from NEW_kr_diff_mc_train.ipynb
def NEW_SiPM_map_plot(xarr, normalize=True):
    """
    Plots a SiPM map in the NEW Geometry
    xarr is a NEW sipm map, yarr the pair of coordinates the map corresponds to
    """
    if normalize:
        probs = (xarr - np.min(xarr))
        probs /= np.max(probs)
    else: 
        probs = xarr

    # set up the figure
    fig = plt.figure();
    ax1 = fig.add_subplot(111);
    fig.set_figheight(5.0)
    fig.set_figwidth(5.0)
    ax1.axis([-100, 100, -100, 100]);

    for i in range(20):
        for j in range(20):
            r = Ellipse(xy=(i * 10 - 95, j * 10 - 95), width=2., height=2.);
            r.set_facecolor('0');
            r.set_alpha(probs[i, j]);
            ax1.add_artist(r);
        
    plt.xlabel("x (mm)");
    plt.ylabel("y (mm)");

## Load in the data

In [ ]:
# read in the training data
x_train, y_train = read_data(data_location, run_name, train_fstart, train_fend)

In [ ]:
# plot one slice of one event
NEW_SiPM_map_plot(x_train[0,:,:,18,0])

## Define and train the DNN

In [ ]:
# set load_model to true and specify the file to load in a previously defined/trained model
load_model = False
mfile = 'models/conv3d_classifier.h5'

if(load_model):
    model = load_model(mfile)
else:
    
    # otherwise define the model
    inputs = Input(shape=(xdim, ydim, zdim, 1))
    model = model_FC(inputs)
    
    # define callbacks (actions to be taken after each epoch of training)
    file_lbl = "{epoch:02d}-{loss:.4f}"
    filepath="weights-{0}.h5".format(file_lbl)
    checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    lcallbacks = [checkpoint]            
    model.summary()

In [ ]:
# train the model
hist = model.fit(x_train, y_train, shuffle=True, nb_epoch=60, batch_size=100, verbose=1, validation_split=0.05, callbacks=lcallbacks)

## Test the trained model

In [ ]:
# read in the test data
x_test, y_test = read_data(data_location, run_name, 3, 5)

In [ ]:
# compute the predictions
loss_and_metrics = model.evaluate(x_test, y_test);
y_pred = model.predict(x_test, batch_size=100, verbose=0)
print(loss_and_metrics)